# Web Scraping with the Art of Literary Text Analysis

Our objective here is to create a local copy of all the articles in journal _Digital Humanities Quarterly_. As always, there are multiple ways to accomplish this, but this will be our plan:

* find a page that lists all the articles (since one of those conveniently exists)
* fetch its contents
* create a list of URLs for the titles
* loop through each URL and save the contents locally

So we'll begin by fetching the URL, something we already saw in the [Getting Started](GettingStarted.ipynb). We begin by importing the built-in [urllib.request](https://docs.python.org/3/library/urllib.request.html#module-urllib.request) library, define the URL to fetch, and fetch it (as HTML source code, so plain text).

In [104]:
import urllib.request

titlesUrl = "http://digitalhumanities.org/dhq/index/title.html" # define the URL
titlesSource = urllib.request.urlopen(titlesUrl).read() # fetch the source from the URL
titlesSource # preview the contents from the URL

b'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE html\n  PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml"><head><meta http-equiv="content-type" content="text/html; charset=utf-8"/><title>DHQ: Digital Humanities Quarterly: Title Index</title><link rel="stylesheet" type="text/css" href="/dhq/common/css/dhq.css"/><link rel="stylesheet" type="text/css" media="screen" href="/dhq/common/css/dhq_screen.css"/><link rel="stylesheet" type="text/css" media="print" href="/dhq/common/css/dhq_print.css"/><link rel="alternate" type="application/atom+xml" href="/dhq/feed/news.xml"/><link rel="shortcut icon" href="/dhq/common/images/favicon.ico"/><script type="text/javascript" src="/dhq/common/js/javascriptLibrary.js">\n                &lt;!-- Javascript functions --&gt;\n            </script><script type="text/javascript">\n\n var _gaq = _gaq || [];\n _gaq.push([\'_setAccount\', \'UA-15812721-1\']);\n _gaq.pus

We have our source code. The next step is to find the list of titles in the document.

In [105]:
from bs4 import BeautifulSoup

titlesSoup = BeautifulSoup(titlesSource) # parse the source document
titleIndex = titlesSoup.find(id="titleIndex") # find the tag with ID=titleIndex 

The next step will be to compile a set of URLs but before we do that we'll detour to explain the concept of lists loops in Python.

A list in Python is an ordered set of elements, sometimes called an array in other languages. A list can be a list of just about any data type, including strings, numbers, objects, and even other lists.

One very easy way to create a list in Python is using the square brackets.

In [106]:
numbers = [0, 1, 2, 3, 4, 5] # new list of numbers
numbers # preview the list

[0, 1, 2, 3, 4, 5]

Now let's say we want to do something with each element in the list, that's called iterating or looping. In Python, as with many languages, there's the concept of a [for loop](https://docs.python.org/3/tutorial/controlflow.html?highlight=loop#for-statements). In this example we'll create tens from our single digits.

In [107]:
tens = [] # create an empty list for our tens
for number in numbers: # for each number in our numbers list
    tens.append(number*10) # add to our tens list the number times ten
tens # preview the output

[0, 10, 20, 30, 40, 50]

Another useful concept is that of the [conditional structure](http://en.wikibooks.org/wiki/Python_Programming/Conditional_Statements) in Python where we test for a boolean value (true or false).

Python uses a colon and <a href="Glossary.ipynb#Indentation" title="Empty spaces used as a formatting tool to designate blocks of code in programming." >indentation </a> to indicate the parts of the <a href="Glossary.ipynb#conditionalblock" title="Where the program has to make a decision based on a series of options using conditional statements" >conditional block. </a> If we want to execute a block when a condition evaluates to true (like ```1 < 5```, one _is_ smaller than five):

    if _condition_:
        _block_

Or if a condition is not true (like ```1 > 5```, one _is not_ smaller than five):

    if *not* _condition_:
        _block_


Using this syntax we can determine if one string is in another.

In [108]:
if "dhq" in titlesUrl:
    print("dhq is in ", titlesUrl)
if not "qhd" in titlesUrl:
    print("qhd is not in ", titlesUrl)

dhq is in  http://digitalhumanities.org/dhq/index/title.html
qhd is not in  http://digitalhumanities.org/dhq/index/title.html


Now that we've introduced lists, conditionals and the "in" operator we can understand the code below more easily. This code uses the BeautifulSoup [find_all](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all) function to create a list of all the &lt;a&gt; (anchor or link) in our document.

In [109]:
urls = [] # create a list of URLs
anchors = titleIndex.find_all("a") # create a list from finding all a (link) tags
for a in anchors: # loop through each element of the anchors list
    href = a['href'] # determine what the href (url) is for the a tag using list subscripting (explained later)
    if "/dhq/vol/" in href: # ensure that we have a proper look URL (not an #anchor)
        urls.append(href) # add the href to the urls list
urls # preview

['/dhq/vol/12/1/000354/000354.html',
 '/dhq/vol/4/1/000084/000084.html',
 '/dhq/vol/3/1/000021/000021.html',
 '/dhq/vol/8/4/000187/000187.html',
 '/dhq/vol/7/1/000142/000142.html',
 '/dhq/vol/8/4/000195/000195.html',
 '/dhq/vol/11/3/000318/000318.html',
 '/dhq/vol/1/2/000010/000010.html',
 '/dhq/vol/12/1/000372/000372.html',
 '/dhq/vol/10/1/000240/000240.html',
 '/dhq/vol/12/1/000367/000367.html',
 '/dhq/vol/12/1/000367/000367.html',
 '/dhq/vol/10/2/000250/000250.html',
 '/dhq/vol/1/2/000011/000011.html',
 '/dhq/vol/10/3/000267/000267.html',
 '/dhq/vol/9/3/ 000227 / 000227 .html',
 '/dhq/vol/10/4/000269/000269.html',
 '/dhq/vol/9/2/000213/000213.html',
 '/dhq/vol/2/1/000019/000019.html',
 '/dhq/vol/10/4/000271/000271.html',
 '/dhq/vol/11/2/000309/000309.html',
 '/dhq/vol/5/3/000100/000100.html',
 '/dhq/vol/3/3/000066/000066.html',
 '/dhq/vol/7/2/000158/000158.html',
 '/dhq/vol/5/1/000101/000101.html',
 '/dhq/vol/8/3/000189/000189.html',
 '/dhq/vol/9/4/000234/000234.html',
 '/dhq/vol/12

As a digression, there's a more "Pythonesque" way of doing things, which is to use [list comprehension](https://docs.python.org/3/tutorial/datastructures.html?highlight=list%20comprehension#list-comprehensions). This is a syntax that's very compact and convenient (though arguable not as legible). The entire code block for looping above can be accomplished in one line. The two approaches are identical (in this case) and you should be familiar with both syntaxes.

In [110]:
urls = [a['href'] for a in titleIndex.find_all("a") if "/dhq/vol/" in a['href']]
urls

['/dhq/vol/12/1/000354/000354.html',
 '/dhq/vol/4/1/000084/000084.html',
 '/dhq/vol/3/1/000021/000021.html',
 '/dhq/vol/8/4/000187/000187.html',
 '/dhq/vol/7/1/000142/000142.html',
 '/dhq/vol/8/4/000195/000195.html',
 '/dhq/vol/11/3/000318/000318.html',
 '/dhq/vol/1/2/000010/000010.html',
 '/dhq/vol/12/1/000372/000372.html',
 '/dhq/vol/10/1/000240/000240.html',
 '/dhq/vol/12/1/000367/000367.html',
 '/dhq/vol/12/1/000367/000367.html',
 '/dhq/vol/10/2/000250/000250.html',
 '/dhq/vol/1/2/000011/000011.html',
 '/dhq/vol/10/3/000267/000267.html',
 '/dhq/vol/9/3/ 000227 / 000227 .html',
 '/dhq/vol/10/4/000269/000269.html',
 '/dhq/vol/9/2/000213/000213.html',
 '/dhq/vol/2/1/000019/000019.html',
 '/dhq/vol/10/4/000271/000271.html',
 '/dhq/vol/11/2/000309/000309.html',
 '/dhq/vol/5/3/000100/000100.html',
 '/dhq/vol/3/3/000066/000066.html',
 '/dhq/vol/7/2/000158/000158.html',
 '/dhq/vol/5/1/000101/000101.html',
 '/dhq/vol/8/3/000189/000189.html',
 '/dhq/vol/9/4/000234/000234.html',
 '/dhq/vol/12

Before saving a local copy of our articles we'll create a data directory to which we can save our files

In [111]:
import os

directory = "data" # data directory
if not os.path.exists(directory): # if the directory doesn't exist
    os.makedirs(directory) # create it

Now we can go ahead and fetch the URLs, writing out the results as we go along. Some concepts won't be explained immediately, such as the [regular expressions](https://docs.python.org/3.7/library/re.html) (`re`) and the `with open` syntax.

In [113]:
import re
import time

for url in set(urls): # for each url in our list of URLs
    clean_url = re.sub(r'\s+', "", url) # clean our URLs of superfluous spaces
    filename = os.path.basename(clean_url) # determine the filename
    path = os.path.join(directory, filename) # construct a full path
    if os.path.exists(path): # if we alredy have the path locally
        print("already fetched:", clean_url) # let the user know we've 
    else: # otherwise we need to fetch it
        print("fetching:", clean_url) # let the user know we've 
        contents = urllib.request.urlopen(urlRoot+clean_url).read() # grab the contents of the URL
        with open(path, "wb") as f: # open the file for writing
            f.write(contents) # write the contents
        time.sleep(1) # be nice to the server by waiting

fetching: /dhq/vol/7/1/000153/000153.html
fetching: /dhq/vol/9/2/000208/000208.html
fetching: /dhq/vol/6/2/000121/000121.html
fetching: /dhq/vol/11/3/000321/000321.html
fetching: /dhq/vol/3/4/000078/000078.html
fetching: /dhq/vol/11/3/000331/000331.html
fetching: /dhq/vol/5/3/000098/000098.html
fetching: /dhq/vol/9/4/000220/000220.html
fetching: /dhq/vol/10/2/000254/000254.html
fetching: /dhq/vol/2/1/000019/000019.html
fetching: /dhq/vol/3/3/000053/000053.html
fetching: /dhq/vol/8/2/000180/000180.html
fetching: /dhq/vol/12/4/000403/000403.html
fetching: /dhq/vol/5/1/000101/000101.html
fetching: /dhq/vol/6/2/000139/000139.html
fetching: /dhq/vol/1/1/000005/000005.html
fetching: /dhq/vol/5/3/000107/000107.html
fetching: /dhq/vol/11/3/000310/000310.html
fetching: /dhq/vol/11/4/000328/000328.html
fetching: /dhq/vol/3/1/000022/000022.html
fetching: /dhq/vol/12/1/000346/000346.html
fetching: /dhq/vol/11/3/000333/000333.html
fetching: /dhq/vol/9/4/000226/000226.html
fetching: /dhq/vol/12/1/00

fetching: /dhq/vol/8/1/000172/000172.html
fetching: /dhq/vol/12/2/000376/000376.html
fetching: /dhq/vol/8/1/000168/000168.html
fetching: /dhq/vol/9/3/000214/000214.html
fetching: /dhq/vol/2/1/000015/000015.html
fetching: /dhq/vol/8/4/000197/000197.html
fetching: /dhq/vol/11/2/000298/000298.html
fetching: /dhq/vol/11/3/000323/000323.html
fetching: /dhq/vol/3/1/000032/000032.html
fetching: /dhq/vol/9/2/000211/000211.html
fetching: /dhq/vol/7/1/000143/000143.html
fetching: /dhq/vol/10/3/000267/000267.html
fetching: /dhq/vol/11/4/000342/000342.html
fetching: /dhq/vol/1/1/000001/000001.html
fetching: /dhq/vol/3/3/000060/000060.html
fetching: /dhq/vol/6/3/000130/000130.html
fetching: /dhq/vol/10/2/000242/000242.html
fetching: /dhq/vol/12/2/000379/000379.html
fetching: /dhq/vol/11/4/000358/000358.html
fetching: /dhq/vol/11/2/000317/000317.html
fetching: /dhq/vol/11/3/000311/000311.html
fetching: /dhq/vol/10/4/000271/000271.html
fetching: /dhq/vol/8/3/000183/000183.html
fetching: /dhq/vol/6/2/

fetching: /dhq/vol/10/1/000238/000238.html
fetching: /dhq/vol/6/3/000135/000135.html
fetching: /dhq/vol/11/3/000332/000332.html
fetching: /dhq/vol/11/1/000274/000274.html
fetching: /dhq/vol/12/2/000394/000394.html
fetching: /dhq/vol/3/2/000047/000047.html
fetching: /dhq/vol/6/1/000110/000110.html
fetching: /dhq/vol/12/2/000390/000390.html
fetching: /dhq/vol/1/2/000011/000011.html
fetching: /dhq/vol/11/2/000288/000288.html


And there we are, we now have a local copy of all the articles from _Digital Humanities Quarterly_!

---
[CC BY-SA](https://creativecommons.org/licenses/by-sa/4.0/) From [The Art of Literary Text Analysis](ArtOfLiteraryTextAnalysis.ipynb) by [Stéfan Sinclair](http://stefansinclair.name) &amp; [Geoffrey Rockwell](http://geoffreyrockwell.com). <br >Created January 24, 2019 (Jupyter 5)